In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [2]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF version: 2.0.0-rc1
Hub version: 0.6.0
GPU is NOT AVAILABLE


In [3]:
module_selection = ("mobilenet_v2", 224) #@param ["(\"mobilenet_v2\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

BATCH_SIZE = 32 #@param {type:"integer"}

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224)


In [7]:
# import shutil
# shutil.rmtree('/root/.keras/datasets/km/coax')
# os.listdir('/root/.keras/datasets/')

FileNotFoundError: ignored

In [9]:
test_folder = 'test11'
data_dir = tf.keras.utils.get_file(
    fname='{}.zip'.format(test_folder),
    origin='https://drive.google.com/uc?export=download&id=1MbP_DNTWVuRkUOK_cWMsqmNwu-xzPB-Y'
    )
# https://drive.google.com/open?id=1MbP_DNTWVuRkUOK_cWMsqmNwu-xzPB-Y
# coaxid = 1Vxlmmck0TKMy-aDRNy9h9v4De1veMBpw
# https://drive.google.com/open?id=1Vxlmmck0TKMy-aDRNy9h9v4De1veMBpw
import zipfile
with zipfile.ZipFile('/root/.keras/datasets/{}.zip'.format(test_folder), 'r') as zip_ref:
    zip_ref.extractall('/root/.keras/datasets/km')
print(os.listdir('/root/.keras/datasets/km/'))

23805952/Unknown - 5s 0us/step['__MACOSX', 'coax', 'ethernet_cable']


In [11]:
# import shutil
etest = 'ethernet_cable'
print(os.path.exists('/root/.keras/datasets/km/{}'.format(etest)))

# shutil.rmtree('/root/.keras/datasets/km/__MACOSX')
i = 1

# os.mkdir('/root/.keras/datasets/km/{}'.format(etest))
print(os.listdir('/root/.keras/datasets/km/{}'.format(etest)))

# for filename in os.listdir('/root/.keras/datasets/km/{}'.format(etest)):
#   print(filename)
#   os.rename('/root/.keras/datasets/km/{etest}/{filename}'.format(etest=etest, filename=filename), '/root/.keras/datasets/km/{etest}/{new_filename}'.format(etest=etest, new_filename=str(i)+'.jpg'))
#   print(i)
#   i+=1



# print(data_dir)

True
['50cm Grey CAT6 Network Cable RCM Certified Ethernet LAN ....jpg', 'ethernet_1569099654029_13.jpg', 'ethernet_1569099654028_5.jpg', '25FT CAT5 MODEM CABLE BLACK ETHERNET LAN NETWORK RJ45 WIRE ....jpg', 'Gray RJ45 Cat5e Ethernet LAN Network Cable ΓÇô 3 feet.jpg', 'Passive Power Over Ethernet Cable Kit - from LINDY UK.jpg', 'CAT6 1000FT UTP SOLID NETWORK ETHERNET CABLE BULK WIRE ....jpg', 'ethernet_1569099654033_42.jpg', 'Cat 6 UTP LAN Blue Network Cable with stranded 568 B wire ....jpg', 'ethernet_1569099654029_15.jpg', '4K UHD HDMI High Speed with Ethernet Cable, 50ft.jpg', 'ethernet_1569099654033_49.jpg', '35ft Flat Cat5e White Ethernet Patch Cable, Molded Boot.jpg', '1000ft Shielded Blue Cat6 Ethernet Spool, Solid, Bulk.jpg', '20ft Cat5e Blue Ethernet Patch Cable, Bootless.jpg', 'High Speed HDMI Cable with Ethernet and 3D ΓÇô 25Feet.jpg', '1000ft Shielded Gray Cat6 Ethernet Spool, Solid, Bulk.jpg', 'Can I I use a cat6 network cable instead of a cat5e cable ....jpg', 'Network Ca

In [12]:
# for filename in os.listdir('/root/.keras/datasets/km/{}'.format(etest)):
#   print(filename)

import shutil

# print(os.listdir('/root/.keras/datasets/km/'))
shutil.rmtree('/root/.keras/datasets/km/__MACOSX')
# shutil.rmtree('/root/.keras/datasets/km/ethernet')
print(os.listdir('/root/.keras/datasets/km/'))


['coax', 'ethernet_cable']


In [13]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

print(os.listdir('/root/.keras/datasets/km/'))
data_dir = '/root/.keras/datasets/km/'

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation = True #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

['coax', 'ethernet_cable']
Found 62 images belonging to 2 classes.
Found 248 images belonging to 2 classes.


In [0]:
do_fine_tuning = True #@param {type:"boolean"}

In [15]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2562      
Total params: 2,260,546
Trainable params: 2,226,434
Non-trainable params: 34,112
_________________________________________________________________


In [0]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
  metrics=['accuracy'])

In [17]:
print(train_generator.samples, valid_generator.samples)
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
hist = model.fit_generator(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps).history


248 62
Epoch 1/5


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


6/7 [========================>.....] - ETA: 11s - loss: 0.7814 - accuracy: 0.6091WARNING:tensorflow:5 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7fc250c0e620> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


7/7 [==============================] - 84s 12s/step - loss: 0.7683 - accuracy: 0.6293 - val_loss: 0.6415 - val_accuracy: 0.7188
Epoch 2/5
7/7 [==============================] - 77s 11s/step - loss: 0.4905 - accuracy: 0.8939 - val_loss: 0.4233 - val_accuracy: 0.9375
Epoch 3/5
7/7 [==============================] - 72s 10s/step - loss: 0.4395 - accuracy: 0.9418 - val_loss: 0.3898 - val_accuracy: 0.9688
Epoch 4/5
7/7 [==============================] - 75s 11s/step - loss: 0.4148 - accuracy: 0.9448 - val_loss: 0.5421 - val_accuracy: 0.8438
Epoch 5/5
7/7 [==============================] - 77s 11s/step - loss: 0.3994 - accuracy: 0.9419 - val_loss: 0.5468 - val_accuracy: 0.8750


In [23]:
saved_model_path = "/tmp/saved_flowers_model"
tf.saved_model.save(model, saved_model_path)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()

with open('/tmp/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/saved_flowers_model/assets


INFO:tensorflow:Assets written to: /tmp/saved_flowers_model/assets


In [21]:
#@title Optimization settings
optimize_lite_model = True  #@param {type:"boolean"}
#@markdown Setting a value greater than zero enables quantization of neural network activations. A few dozen is already a useful amount.
num_calibration_examples = 60  #@param {type:"slider", min:0, max:1000, step:1}
representative_dataset = None
if optimize_lite_model and num_calibration_examples:
  # Use a bounded number of training examples without labels for calibration.
  # TFLiteConverter expects a list of input tensors, each with batch size 1.
  representative_dataset = lambda: itertools.islice(
      ([image[None, ...]] for batch, _ in train_generator for image in batch),
      num_calibration_examples)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
if optimize_lite_model:
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  if representative_dataset:  # This is optional, see above.
    converter.representative_dataset = representative_dataset
lite_model_content = converter.convert()

with open("/tmp/lite_cable_model", "wb") as f:
  f.write(lite_model_content)
print("Wrote %sTFLite model of %d bytes." %
      ("optimized " if optimize_lite_model else "", len(lite_model_content)))

Wrote optimized TFLite model of 2722784 bytes.


In [0]:
interpreter = tf.lite.Interpreter(model_content=lite_model_content)
# This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
def lite_model(images):
  interpreter.allocate_tensors()
  interpreter.set_tensor(interpreter.get_input_details()[0]['index'], images)
  interpreter.invoke()
  return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

In [0]:
#@markdown For rapid experimentation, start with a moderate number of examples.
num_eval_examples = 50  #@param {type:"slider", min:0, max:700}
eval_dataset = ((image, label)  # TFLite expects batch size 1.
                for batch in train_generator
                for (image, label) in zip(*batch))
count = 0
count_lite_tf_agree = 0
count_lite_correct = 0
for image, label in eval_dataset:
  probs_lite = lite_model(image[None, ...])[0]
  probs_tf = model(image[None, ...]).numpy()[0]
  y_lite = np.argmax(probs_lite)
  y_tf = np.argmax(probs_tf)
  y_true = np.argmax(label)
  count +=1
  if y_lite == y_tf: count_lite_tf_agree += 1
  if y_lite == y_true: count_lite_correct += 1
  if count >= num_eval_examples: break
print("TF Lite model agrees with original model on %d of %d examples (%g%%)." %
      (count_lite_tf_agree, count, 100.0 * count_lite_tf_agree / count))
print("TF Lite model is accurate on %d of %d examples (%g%%)." %
      (count_lite_correct, count, 100.0 * count_lite_correct / count))

TF Lite model agrees with original model on 48 of 50 examples (96%).
TF Lite model is accurate on 45 of 50 examples (90%).


In [0]:
print(os.listdir('/tmp'))

['tfhub_modules', 'saved_flowers_model', 'lite_flowers_model', 'pymp-yuycw5oe', 'tmptpii1h7k']


In [0]:
from google.colab import drive
# drive.mount('/content/gdrive')
# converter.convert().save("/content/gdrive/My Drive/wiresliterealmodel")
open('/content/gdrive/My Drive/wiresliterealmodel/goodgoodmodel.tflite', 'wb').write(converter.convert())

FileNotFoundError: ignored